In [1]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install tiktoken openai --quiet --upgrade

Note: you may need to restart the kernel to use updated packages.


# Managing Conversation State with OpenAI's API
In this notebook, we'll walk through how to manage conversation state when interacting with the OpenAI API manually. We'll cover two approaches:

### Manually managing conversation state: 
Building a conversation history list and appending messages for each turn.

### Chaining responses using previous_response_id: 
This method enables the model to access prior context automatically.

----


## 1. Manual Conversation State Management
For this first example, we'll create a simple knock-knock joke conversation by manually constructing a conversation history. Each turn of the conversation is stored in a list of messages, where each message is a dictionary containing the role (user or assistant) and its content.

This approach to state management is explicit and works well when you need fine-grained control over the conversation history.

In [5]:
from dotenv import load_dotenv
import os
from openai import OpenAI

In [6]:
MODEL = "gpt-4.1-mini"

In [7]:

# Load environment variables from .env file
load_dotenv()

client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY")
)

In [8]:
# Construct a conversation history manually for a knock-knock joke
history = [
    {"role": "user", "content": "knock knock."},
    {"role": "assistant", "content": "Who's there?"},
    {"role": "user", "content": "Orange."}
]

# Create a response using the conversation history
response = client.responses.create(
    model=MODEL,
    input=history
)

print("Assistant response:", response.output_text)

# (Optional) Inspect the full response object for further details
# print(response)

Assistant response: Orange who?


## 2. Updating Conversation History Across Rounds

In this example, we'll start a conversation with a simple joke prompt. We'll then update the conversation history with the model's response and ask for another joke. Updating the conversation history manually ensures the model has a full context of previous turns.

Let's see how this works.

In [10]:
# Start a new conversation with an initial joke request
history = [
    {"role": "user", "content": "tell me a joke"}
]

response = client.responses.create(
    model=MODEL,
    input=history,
    store=False  # We're not storing the conversation automatically
)

print("Joke:", response.output_text)

# Update the conversation history with all parts of the response
history += [{"role": el.role, "content": el.content} for el in response.output]

# Ask for another joke in the same conversation
history.append({"role": "user", "content": "tell me another"})

second_response = client.responses.create(
    model=MODEL,
    input=history,
    store=False
)

print("Another joke:", second_response.output_text)

Joke: Sure! Here's a joke for you:

Why don't scientists trust atoms?

Because they make up everything!
Another joke: Of course! Here's another one:

Why did the scarecrow win an award?

Because he was outstanding in his field!


## 3. Chaining Responses with previous_response_id

Another way to carry conversation context is to chain responses by using the previous_response_id parameter. With this method, you initiate a conversation and then provide additional requests that reference the previous response.

Below is an example where we first ask the model for a joke and then ask it to explain why that joke is funny.

In [11]:
# Ask for a joke
response = client.responses.create(
    model=MODEL,
    input="tell me a joke"
)

print("Joke:", response.output_text)

# Chain the next request using previous_response_id so the model retains context
second_response = client.responses.create(
    model=MODEL,
    previous_response_id=response.id,
    input=[{"role": "user", "content": "explain why this is funny."}]
)

print("Explanation:", second_response.output_text)

Joke: Sure! Here's a joke for you:

Why don't skeletons fight each other?

Because they don't have the guts!
Explanation: Absolutely! This joke is funny because it uses a play on words involving a double meaning of the phrase "don't have the guts."

1. **Literal meaning:** Skeletons literally don't have guts—they are just bones without any internal organs like intestines or stomach.

2. **Figurative meaning:** The phrase "don't have the guts" is an idiom meaning "don't have the courage" to do something.

So, the joke plays on the fact that skeletons both physically lack guts and, humorously, would therefore lack the courage to fight. The surprise comes from linking the literal anatomical fact with the figurative expression in a clever way. This clever wordplay creates the humor.
